In [54]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
import numpy as np
import random as rd

PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)

def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

def generateMAFile(template, valFile, N, M, atomicos, inputs, links_internal, links_external):
    templateMA = template
    context = { 
        'N' : N,
        'M' : M,
        'valFile'           : valFile,
        'atomicos'   : atomicos,
        'inputs'            : inputs, 
        'links_internal'    : links_internal, 
        'links_external'    : links_external 
    }
    maFile = render_template(templateMA, context)
    return maFile

def generateAtomicFile(templateH, templateCPP, atomicClass, outPorts, numberOfChosenOutputPorts, outValue):
    context = {
        'atomicClass': atomicClass,
        'atomicClassConstant' : atomicClass.upper(),
        'outPorts'   : outPorts,
        'numberOfOutputPorts' : len(outPorts),
        'numberOfChosenOutputPorts' : numberOfChosenOutputPorts,
        'outValue' : outValue
    }
    hFile   = render_template(templateH, context)
    cppFile = render_template(templateCPP, context)
    return {
        'h'   : hFile,
        'cpp' : cppFile
    }
    return 0

def generateInputAndLinks(N,M,numGroups,modelName,atomicName,atomicClass):
    # Celdas
    cells  = [(n,m) for n in range(0,N) for m in range(0,M)]
    assign = []
    val = 0
    while len(cells) > 0:
        i = rd.randint(0,len(cells)-1)
        assign.append((cells[i][0], cells[i][1], val))
        cells.pop(i)
        val = (val + 1) % numGroups

    # Mapeo conexiones de atomicos 0..numGroups a las celdas correspondientes
    cntOut = [0] * numGroups
    outputsPerAtomic = int(N * M / numGroups)
    assert(outputsPerAtomic * numGroups == N * M)
    
    outputsAtomicos = [[] for i in range(0,numGroups)]
    
    inputsModelo  = []
    linksInternos = []
    linksExternos = []
    for assignation in assign:
        inputCellNumber = str(assignation[0]) + str(assignation[1])
        
        # Inputs modelo
        inputsModelo.append('in' + inputCellNumber)

        # Links internos
        linksInternos.append('link : in' + inputCellNumber + ' in@' + modelName + 
                             '(' + str(assignation[0]) + ',' + str(assignation[1]) + ',0)')
        # Links externos
        outputAtomicNumber = str(assignation[2])
        linksExternos.append('link : out' + inputCellNumber + '@' + atomicName + outputAtomicNumber +
                             ' ' + 'in' + inputCellNumber + 
                             '@' + modelName)

        # Mapa Outputs <==> Atomicos
        outputsAtomicos[assignation[2]].append('out' + inputCellNumber)
    
        # Instancias de atomicos
        atomicos = [atomicName + str(i) + '@' + atomicClass + str(i) for i in range(0,numGroups)]
        
        # actualizo valores 
        cntOut[assignation[2]] += 1

    return {'inputsModelo'  : sorted(inputsModelo), 
            'linksInternos' : sorted(linksInternos),
            'linksExternos' : sorted(linksExternos),
            'atomicos'      : atomicos,
            'outputsAtomicos' : outputsAtomicos}

In [56]:
# Parametros
N = 10
M = 5
numGroups = 5
modelName  = 'opinion'
atomicName = 'shock'
atomicClass= 'Shock'

# Genero archivo .ma
inputsLinks = generateInputAndLinks(N,M,numGroups,modelName,atomicName,atomicClass)
inputsModelo   = inputsLinks['inputsModelo']
links_externos = inputsLinks['linksExternos']
links_internos = inputsLinks['linksInternos']
atomicos       = inputsLinks['atomicos']
outputsAtomicos = inputsLinks['outputsAtomicos']


maFilename = 'Influmodel.ma'
mafile = generateMAFile('model_template.ma', 'valfile.val', N, M,
                        atomicos, inputsModelo, 
                        links_internos, links_externos)
folder = './'
with open(folder + maFilename, 'w') as f:
        f.write(mafile)

# Genero archivos .h y .cpp 
folder = 'src/'
for nShock in range(0, numGroups):
    atomicClass = 'Shock' + str(nShock)
    outPorts = outputsAtomicos[nShock]
    nChosenOutports = int(len(outPorts) / 2)
    outValue = 10

    atomic = generateAtomicFile('atomic_template.h', 'atomic_template.cpp', 
                                atomicClass, outPorts, nChosenOutports, outValue)

    with open(folder + atomicClass + '.h', 'w') as f:
        hFile   = atomic['h']
        f.write(hFile)

    with open(folder + atomicClass + '.cpp', 'w') as f:
        cppFile = atomic['cpp']
        f.write(cppFile)

# Genero reg.cpp file
## TODO
